In [0]:
def get_silver_table_names():
    silver_tables_df = spark.sql("""
        SELECT table_name 
        FROM tracking_table 
        WHERE layer = 'Silver'
    """)
    return [row.table_name for row in silver_tables_df.collect()]

In [0]:
def dq_checks_silver(input_df):
    failure_log = []
    
    try:
        # Check for data type conformity
        for col_name, expected_type in expected_types.items():
            actual_type = input_df.schema[col_name].dataType.simpleString
            if actual_type != expected_type:
                failure_log.append((col_name, f"Type mismatch: expected {expected_type}, found {actual_type}"))
        
        # Check for valid values in categorical columns
        invalid_values = input_df.filter(~col("category_column").isin(valid_categories)).collect()
        for row in invalid_values:
            failure_log.append((row["business_key"], "Invalid category value"))
        
        # Log failures to the dq_failure_log table
        if failure_log:
            for record_id, reason in failure_log:
                spark.sql(f"""
                    INSERT INTO dq_failure_log (layer, record_id, failure_reason)
                    VALUES ('Silver', '{record_id}', '{reason}')
                """)
    
    except Exception as e:
        # Log exceptions to the dq_failure_log table
        spark.sql(f"""
            INSERT INTO dq_failure_log (layer, record_id, failure_reason)
            VALUES ('Silver', NULL, 'Exception: {str(e)}')
        """)
    
    return failure_log

In [0]:
def log_to_table(table_name, layer, log_message, error_flag=False):
    timestamp = spark.sql("SELECT current_timestamp()").collect()[0][0]
    spark.sql(f"""
        INSERT INTO logging_table (table_name, layer, log_message, log_timestamp, error_flag)
        VALUES ('{table_name}', '{layer}', '{log_message}', '{timestamp}', {error_flag})
    """)

In [0]:
try:
    # Get the list of silver table names
    silver_table_names = get_silver_table_names()  # Assuming this function exists
    for table_name in silver_table_names:
    # Load the DataFrame for the current silver table
        input_df = spark.table(table_name)

        # Execute the silver data quality checks
        dq_checks_silver(input_df)

except Exception as e:
    log_to_table(table_name, "Silver", f"No configuration found for table: {table_name}", error_flag=True)
    print(f"Error during silver table operations: {str(e)}")